# Pandas - Extracting data

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier, BernoulliRBM
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LassoCV, ElasticNetCV, SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.tree import ExtraTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.preprocessing import PolynomialFeatures
train = pd.read_csv('data/train.csv', sep=';')
test = pd.read_csv('data/test.csv', sep=';', names=list(train))

# Pandas - Cleaning data

In [9]:
frames = [train, test]
data = pd.concat(frames, axis=0)
categorical_columns = list(data.select_dtypes(include=['object']))
#print(categorical_columns)
for col in categorical_columns:
    #print(data[col].unique())
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes
data['y'] = data['y'].replace(to_replace=0, value=-1)

In [40]:
unique, counts = np.unique(data['y'], return_counts=True)
decompte = dict(zip(unique, counts))

In [53]:
pca = PCA(n_components=2, svd_solver='full', whiten=False)
poly = PolynomialFeatures(degree=2, interaction_only=True)
data = data.sample(frac=1).groupby('y', sort=False).head(decompte[1])
#poly_data = 
data.head(10)
#X,y = pca.fit_transform(normalize(data[data.columns[:-1]].values)),data[data.columns[-1]].values
X,y = pca.fit_transform(data[data.columns[:-1]].values),data[data.columns[-1]].values
X_train, X_test = np.split(poly.fit_transform(X), [train.shape[0]])
y_train, y_test = np.split(y, [train.shape[0]])
X_train


array([[  1.00000000e+00,  -2.88624567e+02,  -2.23425435e+02,
          6.44860694e+04],
       [  1.00000000e+00,  -4.22250721e+02,   7.80051901e+02,
         -3.29377478e+05],
       [  1.00000000e+00,  -2.94421063e+02,  -2.24941020e+02,
          6.62273744e+04],
       ..., 
       [  1.00000000e+00,  -4.01790166e+02,   8.01567785e+02,
         -3.22062053e+05],
       [  1.00000000e+00,   1.46897483e+02,  -1.03558974e+02,
         -1.52125526e+04],
       [  1.00000000e+00,   5.13482520e+02,  -7.56774427e+00,
         -3.88590440e+03]])

In [54]:
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
#param_grid = {}
#clf = GridSearchCV(MLPClassifier('solver':'adam', 'activation':'relu'), cv=10, n_jobs=-1, param_grid=param_grid)
#clf = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
#beta_2=0.999, early_stopping=False, epsilon=1e-08,
#hidden_layer_sizes=(100,), learning_rate='invscaling',
#learning_rate_init=0.001, max_iter=10000, momentum=0.9,
#nesterovs_momentum=True, power_t=0.5, random_state=None,
#shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
#verbose=False, warm_start=True)
#clf = AdaBoostClassifier(n_estimators=300)
#clf = BalancedBaggingClassifier(KNeighborsClassifier(n_neighbors=7, leaf_size=25),n_estimators=100)
#param_grid = {'C':np.arange(0.44,0.445,0.0001)}
#clf = GridSearchCV(SVC(kernel='poly',class_weight='balanced',tol=1e-4), cv=10, n_jobs=-1, param_grid=param_grid)
#clf = BaggingClassifier(base_estimator=SVC(C=0.06875,kernel='rbf',class_weight='balanced',tol=1e-4), n_jobs=-1, n_estimators=100)
#clf = BaggingClassifier(LogisticRegression(),n_jobs=-1)
clf = LogisticRegression(C=0.07)
#decompte = dict(zip(unique, counts))
#clf = BalancedBaggingClassifier(AdaBoostClassifier(algorithm='SAMME', base_estimator=SVC(kernel='linear', tol=1e-4, C=3.5, class_weight='balanced', probability=True)), oob_score=True, replacement=True, bootstrap=True, verbose=0, max_samples = 2* dict(zip(unique, counts))[1], ratio = 'majority', n_jobs=-1, random_state=42)
#clf = ElasticNetCV(cv = 10,normalize=True,tol=1e-5, eps=0.01, verbose=0, n_jobs=-1)
#clf = BalancedBaggingClassifier(base_estimator=LogisticRegression(C=0.7, tol=0.00001, class_weight='balanced'), bootstrap=True, verbose=0, max_samples = 2* dict(zip(unique, counts))[1], ratio = 'majority', n_jobs=-1, random_state=42 )
#clf = SGDClassifier(loss='hinge',penalty='elasticnet',shuffle=True, random_state=42, class_weight='balanced')
#clf = RandomForestClassifier(class_weight='balanced', n_estimators=20,criterion='entropy', random_state=42)
#clf = BernoulliRBM()
#clf = MLPClassifier(random_state=42)
#clf = AdaBoostClassifier(base_estimator=KNeighborsClassifier(), random_state=42)
#clf = KNeighborsClassifier()
clf.fit(X_train_train, y_train_train)
y_train_pred = clf.predict(X_train_test)
#y_train_pred[y_train_pred<0] = -1.0
#y_train_pred[y_train_pred>0] = 1.0
#print(clf.best_params_)


In [55]:
print(accuracy_score(y_train_test,y_train_pred))
print(roc_auc_score(y_train_test,y_train_pred))
print(mean_squared_error(y_train_test,y_train_pred))

0.701492537313
0.704174228675
1.19402985075


In [56]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[y_pred<0] = -1.0
y_pred[y_pred>0] = 1.0
y_pred = [int(i) for i in y_pred]
result = np.c_[range(1,len(y_pred)+1), y_pred]
df_result = pd.DataFrame(result[:,0:2], columns=['Id', 'prediction'])
df_result['prediction'] = df_result['prediction'].replace(to_replace=-1, value=0)
#df_result[]
df_result.to_csv('prediction.csv', index=False)

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.